# huggingface 处理大文件

1.1 本地文件处理为dataset

1.2 分块加载大文件

1.3 AI模型如何读取大文件

In [2]:
from datasets import load_dataset

In [12]:
data_files = {"train": "/home/ec2-user/project/AI/Study/Week1/data/train.csv", "validation": "/home/ec2-user/project/AI/Study/Week1/data/val.csv"}
down_dataset = load_dataset("csv", data_files=data_files)

In [13]:
down_dataset # train表 的 第一行

DatasetDict({
    train: Dataset({
        features: ['prot_id', 'seq', 'seq_len', 'pdb_filename', 'ptm', 'mean_plddt', 'emb_filename', 'label', 'source'],
        num_rows: 963
    })
    validation: Dataset({
        features: ['prot_id', 'seq', 'seq_len', 'pdb_filename', 'ptm', 'mean_plddt', 'emb_filename', 'label', 'source'],
        num_rows: 275
    })
})

1.2 分块加载大文件

In [24]:
#!pip install zstandard

In [7]:
from datasets import load_dataset

# 这需要几分钟才能运行,所以在你等待的时候去喝杯茶或咖啡 :)
data_files = "https://huggingface.co/datasets/casinca/PUBMED_title_abstracts_2019_baseline/resolve/main/PUBMED_title_abstracts_2019_baseline.jsonl.zst"
pubmed_dataset_streamed = load_dataset(
    "json", 
    data_files=data_files, 
    split="train",
    streaming=True, # 使用streaming模式来处理大型数据集
    cache_dir="../test/data"
    )
#pubmed_dataset
#next(iter(pubmed_dataset_streamed))


1.3 AI模型如何读取大文件

首先要把数据集转为embedding，然后导入到模型中去训练？

In [ ]:
# 转为 Embedding 向量的代码
from sentence_transformers import SentenceTransformer
import torch

# 1. 加载一个预训练的医疗领域 Embedding 模型 (例如 BioBERT 相关的或专用的 Embedding 模型)
# 如果没有特殊需求，可以使用通用且高效的 'all-MiniLM-L6-v2'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 如果有 GPU，把模型移动到 GPU 上，速度会快几十倍
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def compute_embeddings(batch):
    # 提取摘要字段（注意：根据数据结构确认 key 是 "abstract" 还是其他）
    # “key”在这里对应的就是数据集里的“列名”（Column Name）。
    texts = batch["abstract"]
    
    # 将文本转换为向量
    # convert_to_numpy=True 方便后续存储，show_progress_bar 关掉避免流式报错
    embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    
    # 将向量放回 batch 中返回
    return {"embeddings": embeddings}

# 2. 使用 map 进行流式转换
# batched=True 表示一次处理多条数据（这里设为 32 条），这比一条条处理快得多
pubmed_with_embeddings = pubmed_dataset_streamed.map(
    compute_embeddings, 
    batched=True, 
    batch_size=32 # 与显存有关
)

# 3. 验证第一条数据的向量
first_record = next(iter(pubmed_with_embeddings))
print(f"标题: {first_record['title']}")
print(f"向量维度: {len(first_record['embeddings'])}")
print(f"向量预览: {first_record['embeddings'][:5]}")

ModuleNotFoundError: No module named 'sentence_transformers'

In [15]:
# 保存到本地的代码
#存为 Parquet 文件（适合后续模型训练）
import os
import pandas as pd

# 创建存放目录
save_dir = "./pubmed_embeddings"
os.makedirs(save_dir, exist_ok=True)

# 假设我们每 1000 条保存一个文件，防止内存溢出
batch_size = 1000
current_batch = []
file_count = 0

print("开始处理并保存...")

# 这里的 pubmed_with_embeddings 是我们上一段代码中 map 后的流式对象
for i, record in enumerate(pubmed_with_embeddings):
    current_batch.append(record)
    
    # 当累计到指定条数时，保存到本地
    if (i + 1) % batch_size == 0:
        df = pd.DataFrame(current_batch)
        file_path = os.path.join(save_dir, f"part_{file_count}.parquet")
        df.to_parquet(file_path)
        
        print(f"已保存 {i+1} 条数据到 {file_path}")
        current_batch = []
        file_count += 1
        
        # 仅作为演示：如果你只想测试前 5000 条，可以 break
        if file_count >= 5: 
            break

# 处理剩余的数据
if current_batch:
    df = pd.DataFrame(current_batch)
    df.to_parquet(os.path.join(save_dir, f"part_{file_count}.parquet"))

开始处理并保存...


NameError: name 'pubmed_with_embeddings' is not defined

HDF5：存储纯数字非常快，但如果你想同时存“论文标题（字符串）”、“作者（列表）”和“向量（数组）”，HDF5 会变得非常麻烦，且效率降低。